# Estudio de eventos en mercados de criptomonedas

#### Librerías y versiones:

In [1]:
import copy
import datetime as dt
import numpy as np
import pandas as pd 
import pandas_datareader as pdr
import statsmodels.regression.linear_model as lm

- python: 3.8.3
- copy: built-in
- datetime: built-in
- numpy: 1.18.5
- pandas: 1.0.5
- pandas_datareader: 0.9.0
- statsmodels: 0.11.1

### Función para estudiar un evento:
Recibe una fecha y devuelve una lista con tres DataFrames según el modelo de rentabilidad utilizado:
   - Evento estudiado con el Constant Mean Return Model.
   - Evento estudiado con el Market-adjusted Model.
   - Evento estudiado con el Market Model.

Cada uno contiene los excesos de rentabilidad (AR), excesos de rentabilidad acumulada (CAR) y pruebas t de Student (T-stat).

La ventana del evento va desde dos días antes del evento hasta dos días después.

La ventana de estimación está compuesta por los 100 días anteriores a la ventana del evento.

In [2]:
def event_study(date):
    
    # fechas de evento y ventanas
    event = pd.to_datetime(date)
    anticipation = event - dt.timedelta(2)
    adjustment = event + dt.timedelta(2)
    estimation = adjustment - dt.timedelta(100)
    
    # descarga de datos de criptomonedas e índice
    symbols_list = ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'ADA-USD']
    cryptos = pdr.DataReader(symbols_list, 'yahoo', estimation)['Open']
    sp500 = pdr.DataReader('^GSPC', 'yahoo', estimation)["Open"]
    
    
    # creación del dataframe
    data = cryptos.join(sp500, how = "left")
    data.columns = ["BTC", "ETH", "DOGE", "ADA", "SP500"]
    data = data.fillna(method = 'ffill')
    data = data.dropna()
    data = data[~data.index.duplicated()]
    
    # cálculo de returns
    data_returns = data/data.shift(1) - 1
    data_returns = data_returns.dropna()

    # cálculo de retornos medios en ventana de estimación
    retornos_est = data_returns.loc[data_returns.index < anticipation]
    mu = retornos_est.mean()
    
    # cálculo del Constant Mean Return Model (CMRM)
    data_returns["CMRM_BTC"] = data_returns["BTC"] - mu["BTC"]
    data_returns["CMRM_ETH"] = data_returns["ETH"] - mu["ETH"]
    data_returns["CMRM_DOGE"] = data_returns["DOGE"] - mu["DOGE"]
    data_returns["CMRM_ADA"] = data_returns["ADA"] - mu["ADA"]
    
    # cálculo del Market-adjustment Model (MAM)
    data_returns["MAM_BTC"] = data_returns["BTC"] - data_returns["SP500"]
    data_returns["MAM_ETH"] = data_returns["ETH"] - data_returns["SP500"]
    data_returns["MAM_DOGE"] = data_returns["DOGE"] - data_returns["SP500"]
    data_returns["MAM_ADA"] = data_returns["ADA"] - data_returns["SP500"]
    
    # cálculo del Market Model (MM)
    retornos = copy.deepcopy(retornos_est)
    retornos['CT'] = 1
    
    MM_BTC = lm.OLS(retornos["BTC"], retornos[["CT", "SP500"]], hasconst = bool).fit()
    beta_BTC, alpha_BTC = MM_BTC.params["CT"], MM_BTC.params["SP500"] / 100
    
    MM_ETH = lm.OLS(retornos["ETH"], retornos[["CT", "SP500"]], hasconst = bool).fit()
    beta_ETH, alpha_ETH = MM_ETH.params["CT"], MM_ETH.params["SP500"] / 100
    
    MM_DOGE = lm.OLS(retornos["DOGE"], retornos[["CT", "SP500"]], hasconst = bool).fit()
    beta_DOGE, alpha_DOGE = MM_DOGE.params["CT"], MM_DOGE.params["SP500"] / 100
 
    MM_ADA = lm.OLS(retornos["ADA"], retornos[["CT", "SP500"]], hasconst = bool).fit()
    beta_ADA, alpha_ADA = MM_ADA.params["CT"], MM_ADA.params["SP500"] / 100
    
    data_returns["MM_BTC"] = data_returns["BTC"] - (alpha_BTC + beta_BTC * data_returns["SP500"])
    data_returns["MM_ETH"] = data_returns["ETH"] - (alpha_ETH + beta_ETH * data_returns["SP500"])
    data_returns["MM_DOGE"] = data_returns["DOGE"] - (alpha_DOGE + beta_DOGE * data_returns["SP500"])
    data_returns["MM_ADA"] = data_returns["ADA"] - (alpha_ADA + beta_ADA * data_returns["SP500"])
    
    # cálculo de ARs y CARs
    ar = data_returns.loc[(data_returns.index >= anticipation) & (data_returns.index <= adjustment)]
    ar = ar[["CMRM_BTC", "CMRM_ETH", "CMRM_DOGE", "CMRM_ADA",
             "MAM_BTC", "MAM_ETH", "MAM_DOGE", "MAM_ADA",
             "MM_BTC", "MM_ETH", "MM_DOGE", "MM_ADA"]]
    car = ar.cumsum()
    
    # cálculo de desviaciones estándar y t-statistics
    sd =  data_returns.loc[data_returns.index < anticipation].std()
    sd = sd[["CMRM_BTC", "CMRM_ETH", "CMRM_DOGE", "CMRM_ADA",
             "MAM_BTC", "MAM_ETH", "MAM_DOGE", "MAM_ADA",
             "MM_BTC", "MM_ETH", "MM_DOGE", "MM_ADA"]]
    tstats = ar/sd

    # creación de dataframes
    cmrm = pd.DataFrame()
    cmrm[["AR_BTC", "AR_ETH", "AR_DOGE", "AR_ADA"]] = np.round(ar[["CMRM_BTC", "CMRM_ETH",
                                                                 "CMRM_DOGE", "CMRM_ADA"]] * 100, decimals=2)
    
    cmrm[["CAR_BTC", "CAR_ETH", "CAR_DOGE", "CAR_ADA"]] = np.round(car[["CMRM_BTC", "CMRM_ETH",
                                                                      "CMRM_DOGE", "CMRM_ADA"]] * 100, decimals=2)
    
    cmrm[["T-stat_BTC", "T-stat_ETH", "T-stat_DOGE", "T-stat_ADA"]] = np.round(tstats[["CMRM_BTC", "CMRM_ETH",
                                                                                       "CMRM_DOGE","CMRM_ADA"]], decimals=2)
    cmrm=cmrm[["AR_BTC", "CAR_BTC", "T-stat_BTC", "AR_ETH", "CAR_ETH", "T-stat_ETH",
               "AR_DOGE", "CAR_DOGE", "T-stat_DOGE", "AR_ADA", "CAR_ADA", "T-stat_ADA"]]
    
    mam = pd.DataFrame()
    mam[["AR_BTC", "AR_ETH", "AR_DOGE", "AR_ADA"]] = np.round(ar[["MAM_BTC", "MAM_ETH",
                                                                  "MAM_DOGE", "MAM_ADA"]] * 100, decimals=2)
    
    mam[["CAR_BTC", "CAR_ETH", "CAR_DOGE", "CAR_ADA"]] = np.round(car[["MAM_BTC", "MAM_ETH",
                                                                       "MAM_DOGE","MAM_ADA"]] * 100, decimals=2)
    
    mam[["T-stat_BTC", "T-stat_ETH", "T-stat_DOGE", "T-stat_ADA"]] = np.round(tstats[["MAM_BTC", "MAM_ETH",
                                                                                      "MAM_DOGE", "MAM_ADA"]], decimals=2)
    mam=mam[["AR_BTC", "CAR_BTC", "T-stat_BTC", "AR_ETH", "CAR_ETH", "T-stat_ETH",
               "AR_DOGE", "CAR_DOGE", "T-stat_DOGE", "AR_ADA", "CAR_ADA", "T-stat_ADA"]]

    mm = pd.DataFrame()
    mm[["AR_BTC", "AR_ETH", "AR_DOGE", "AR_ADA"]] = np.round(ar[["MM_BTC", "MM_ETH",
                                                                 "MM_DOGE", "MM_ADA"]] * 100, decimals=2)
    
    mm[["CAR_BTC", "CAR_ETH", "CAR_DOGE", "CAR_ADA"]] = np.round(car[["MM_BTC", "MM_ETH",
                                                                      "MM_DOGE", "MM_ADA"]] * 100, decimals=2)
    
    mm[["T-stat_BTC", "T-stat_ETH", "T-stat_DOGE", "T-stat_ADA"]] = np.round(tstats[["MM_BTC", "MM_ETH",
                                                                                     "MM_DOGE", "MM_ADA"]], decimals=2)
    mm=mm[["AR_BTC", "CAR_BTC", "T-stat_BTC", "AR_ETH", "CAR_ETH", "T-stat_ETH",
               "AR_DOGE", "CAR_DOGE", "T-stat_DOGE", "AR_ADA", "CAR_ADA", "T-stat_ADA"]]
    
    results = (cmrm, mam, mm)
    
    return results

### Ejemplo: 12 de mayo de 2021, Tesla anuncia que deja de aceptar Bitcoin por motivos medioambientales.

In [3]:
evento = event_study("2021-05-12")

#### Constant Mean Return Model

In [4]:
evento[0]

,AR_BTC,CAR_BTC,T-stat_BTC,AR_ETH,CAR_ETH,T-stat_ETH,AR_DOGE,CAR_DOGE,T-stat_DOGE,AR_ADA,CAR_ADA,T-stat_ADA
Date,,,,,,,,,,,,
2021-05-10,-4.73,-4.73,-1.13,-0.52,-0.52,-0.10,-25.07,-25.07,-1.53,-8.80,-8.80,-1.20
2021-05-11,0.95,-3.79,0.23,4.61,4.09,0.93,5.45,-19.62,0.33,5.66,-3.13,0.77
2021-05-12,-12.91,-16.70,-3.08,-9.41,-5.32,-1.90,-24.72,-44.34,-1.51,-12.82,-15.95,-1.75
2021-05-13,-0.71,-17.41,-0.17,-3.97,-9.28,-0.80,20.05,-24.29,1.22,21.12,5.17,2.88
2021-05-14,-0.26,-17.67,-0.06,8.44,-0.85,1.71,10.76,-13.54,0.66,2.36,7.53,0.32


#### Market-adjusted Model

In [5]:
evento[1]

,AR_BTC,CAR_BTC,T-stat_BTC,AR_ETH,CAR_ETH,T-stat_ETH,AR_DOGE,CAR_DOGE,T-stat_DOGE,AR_ADA,CAR_ADA,T-stat_ADA
Date,,,,,,,,,,,,
2021-05-10,-4.55,-4.55,-1.10,0.18,0.18,0.04,-21.35,-21.35,-1.31,-7.47,-7.47,-1.02
2021-05-11,3.40,-1.16,0.82,7.58,7.76,1.56,11.45,-9.90,0.70,9.26,1.79,1.26
2021-05-12,-11.83,-12.99,-2.87,-7.80,-0.05,-1.60,-20.09,-30.00,-1.23,-10.58,-8.79,-1.44
2021-05-13,1.24,-11.75,0.30,-1.50,-1.54,-0.31,25.54,-4.46,1.57,24.22,15.43,3.30
2021-05-14,-0.99,-12.74,-0.24,8.23,6.68,1.69,13.57,9.11,0.83,2.78,18.21,0.38


#### Market Model

In [6]:
evento[2]

,AR_BTC,CAR_BTC,T-stat_BTC,AR_ETH,CAR_ETH,T-stat_ETH,AR_DOGE,CAR_DOGE,T-stat_DOGE,AR_ADA,CAR_ADA,T-stat_ADA
Date,,,,,,,,,,,,
2021-05-10,-5.49,-5.49,-1.31,-0.95,-0.95,-0.19,-26.75,-26.75,-1.63,-7.49,-7.49,-1.02
2021-05-11,0.20,-5.29,0.05,4.20,3.26,0.85,3.85,-22.90,0.23,7.01,-0.47,0.96
2021-05-12,-13.67,-18.96,-3.26,-9.83,-6.57,-1.99,-26.37,-49.27,-1.61,-11.49,-11.96,-1.57
2021-05-13,-1.46,-20.42,-0.35,-4.38,-10.95,-0.89,18.43,-30.84,1.12,22.46,10.50,3.06
2021-05-14,-1.02,-21.45,-0.24,8.00,-2.94,1.62,9.04,-21.80,0.55,3.66,14.16,0.50


Autor: Javier Lafuente Azagra